In [1]:
import pandas as pd 
import numpy as np
import networkx as nx 
import sys
import matplotlib.pyplot as plt
from itertools import combinations
from src.utils import fileio

TREATMENT = "LDA_OCT_5DIZ_27_11_2023_10_10_A4"
path = f"/home/milky/sna/data/processed/0_1_2_angles_matrix/LDA_OCT_5DIZ/{TREATMENT}.csv"
df_angles = pd.read_csv(path, index_col=0)

path = f"/home/milky/sna/data/processed/0_1_1_distances_matrix/LDA_OCT_5DIZ/{TREATMENT}.csv"
df_distances = pd.read_csv(path, index_col=0)

path = f"/home/milky/drosophila-SNA/data/processed/0_0_preproc_data/LDA_OCT_5DIZ/{TREATMENT}"
fly_dict = fileio.load_files_from_folder(path)

df_flies_vel = pd.DataFrame()
for fly_name, fly_path in fly_dict.items():
    df = pd.read_csv(fly_path, index_col=0)
    df_flies_vel[fly_name]=df['vel']

sys.exit()


df_angles = df_angles.T
df_distances = df_distances.T
DISTANCE = 2.5
ANGLE = [0, -180]
threshold_value = 0.33
d = {}
for frame in df_angles.columns:
    df_a_d = pd.concat([df_angles.iloc[:, frame], df_distances.iloc[:, frame]], axis=1)
    df_a_d.columns = ["angle", "distance"]
    
    distance_mask = df_a_d["distance"] <= DISTANCE
    # df_a_d = df_a_d[distance_mask]
    angle_mask = (df_a_d["angle"] <= ANGLE[0]) & (df_a_d["angle"] >= ANGLE[1])
    df_a_d = df_a_d[distance_mask & angle_mask]

    G = nx.DiGraph()
    for r in df_a_d.iterrows():
        nodes, values = r
        n_1, n_2 = nodes.split(" ")
        G.add_edge(n_1, n_2)
    
    connected_components = list(nx.connected_components(G.to_undirected()))
    try:
        largest_component = max(connected_components, key=len)
    except:
        pass

    if len(largest_component) > 3:
        G = G.subgraph(largest_component)
        nodes_degree_1 = [node for node in G.nodes if G.degree(node) == 1]
        two_item_pairs = list(combinations(nodes_degree_1, 2))
        for start_node, end_node in two_item_pairs:
            if nx.has_path(G, start_node, end_node) or nx.has_path(G, end_node, start_node):
                # pos = nx.circular_layout(G) 
                # nx.draw(G, pos, with_labels=True, width=1)

                flies_vel_dict = {}
                valid = True
                for fly_name in largest_component:
                    if valid and df_flies_vel[fly_name][frame] > threshold_value:
                        flies_vel_dict[fly_name] = df_flies_vel[fly_name][frame]
                    else:
                        flies_vel_dict = {}
                        valid = False
                if valid:
                    d.update({frame :flies_vel_dict})

def split_consecutive(lst):
    result = []
    sublist = [lst[0]]

    for i in range(1, len(lst)):
        # Check if the current number is consecutive to the previous one
        if lst[i] == lst[i - 1] + 1:
            sublist.append(lst[i])
        else:
            result.append(sublist)
            sublist = [lst[i]]
    
    result.append(sublist)  # Append the last sublist

    final = []
    for sl in result:
        if len(sl)>6:
            final.append(sl)
    
    return final

# Split the list into consecutive sublists
consecutive_sublists = split_consecutive(list(d.keys()))

SystemExit: 

/home/milky/drosophila-SNA/.env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
total_trains_time = 0
for l in consecutive_sublists:
    print(l[0], len(l))
    
#     total_trains_time += len(l)

# print(total_trains_time)

360 9
377 11
481 35
3947 9
3964 9
4104 18
4171 9
4181 7
4510 7
4534 7
11959 8
